In [ ]:
import src.DetectingRegionInfo as DetectingRegionInfo
import src.DopplerInfo as DopplerInfo

import src.LinesGenerator as LinesGenerator
import src.WAndDopplerGenerator as WAndDopplerGenerator
import src.FeaturesAndLabelsGenerator as FeaturesAndLabelsGenerator

import src.Config as Config
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt



# create info structs from config
detecting_region_info = DetectingRegionInfo.DetectingRegionInfo(
    Config.transmittor_position, Config.receiver_position_1, Config.receiver_position_2, Config.receiver_position_3)
doppler_info = DopplerInfo.DopplerInfo(Config.c, Config.fc, Config.v)



In [ ]:
def visualize_lines(detecting_region_info, lines):
    fig, ax = plt.subplots()
    ax.set_xlim(0, 300)
    ax.set_ylim(0, 150)
    ax.add_patch(Polygon([detecting_region_info.v1,
                 detecting_region_info.v2, detecting_region_info.v3], fill=False))
    ax.add_patch(Polygon([detecting_region_info.v3,
                 detecting_region_info.v2, detecting_region_info.v4], fill=False))

    # Plot the lines
    for i in range(len(lines)):
        line = lines[i]
        ax.plot([point[0] for point in line], [point[1]
                for point in line], "r")

    plt.show()


In [ ]:
# training features and labels

[lines_a, lines_b] = LinesGenerator.generateLines(
    detecting_region_info=detecting_region_info,
    num_of_lines_to_generate=Config.train_set_num,
    step_count_per_line=Config.line_seq_count,
    length_per_step=Config.line_step_length,
    angle_change_limit_per_step=Config.line_step_angle_change
)

[w, doppler] = WAndDopplerGenerator.generateWAndDoppler(
    detecting_region_info=detecting_region_info,
    doppler_info=doppler_info,
    lines_a=lines_a,
    lines_b=lines_b
)

[features, labels] = FeaturesAndLabelsGenerator.generateFeaturesAndLabels(
    detecting_region_info=detecting_region_info,
    lines_a=lines_a,
    w=w,
    doppler=doppler,
    num_of_lines_to_generate=Config.train_set_num,
    step_count_per_line=Config.line_seq_count
)

visualize_lines(detecting_region_info, lines_a)

In [ ]:
# testing features and labels

[lines_a, lines_b] = LinesGenerator.generateLines(
    detecting_region_info=detecting_region_info,
    num_of_lines_to_generate=1,
    step_count_per_line=Config.line_seq_count,
    length_per_step=Config.line_step_length,
    angle_change_limit_per_step=Config.line_step_angle_change
)

[w, doppler] = WAndDopplerGenerator.generateWAndDoppler(
    detecting_region_info=detecting_region_info,
    doppler_info=doppler_info,
    lines_a=lines_a,
    lines_b=lines_b
)

[features, labels] = FeaturesAndLabelsGenerator.generateFeaturesAndLabels(
    detecting_region_info=detecting_region_info,
    lines_a=lines_a,
    w=w,
    doppler=doppler,
    num_of_lines_to_generate=Config.train_set_num,
    step_count_per_line=Config.line_seq_count
)

visualize_lines(detecting_region_info, lines_a)

In [ ]:
import os
import time

current_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
result_folder = "results/" + current_time
os.makedirs(result_folder)

training_results_folder = result_folder + "/training"
os.makedirs(training_results_folder)

testing_results_folder= result_folder + "/testing"
os.makedirs(testing_results_folder)


In [ ]:
# 定义更深的神经网络模型
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from IPython.display import clear_output

import src.Config as cf
import src.TrajectoryDataset as TrajectoryDataset

import src.UavModel as UavModel
using_model = UavModel.UavModel()

# 创建训练和测试数据集
train_dataset = TrajectoryDataset.TrajectoryDataset(features, labels)

# 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=cf.train_batch_size, shuffle=True)

# 使用GPU加速
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = using_model.to(device)

for param in model.parameters():
    print(type(param), param.size())

# 损失函数和优化器s
optimizer = optim.Adam(model.parameters(), lr=cf.learning_rate)
criterion = nn.MSELoss()

# 训练模型
loss_values = []  # 存储损失值以供绘图

for epoch in range(cf.epoch):
    for inputs, coords in train_loader:
        inputs, coords = inputs.to(device), coords.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, coords)
        loss.backward()
        optimizer.step()
    loss_values.append(loss.item())

    # 动态绘图
    clear_output(wait=True)
    plt.figure(figsize=(12, 6))
    plt.plot(loss_values, label="Loss")
    plt.title("Loss Curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.text(epoch, loss.item(), f"Loss: {loss.item()}", ha="right")

    train_result_name = training_results_folder + "/up-to-date-loss.png"
    plt.savefig(train_result_name)
    plt.show()

    # 打印损失信息
    print(f"Epoch [{epoch+1}/{cf.epoch}], Loss: {loss.item()}")

    # # 测试模型
    # model.eval()
    # total_test_loss = 0
    # predicted_coords = []
    # true_coords = []
    # with torch.no_grad():
    #     for inputs, coords in test_loader:
    #         inputs, coords = inputs.to(device), coords.to(device)
    #         outputs = model(inputs)
    #         test_loss = criterion(outputs, coords)
    #         total_test_loss += test_loss.item()
    #         predicted_coords.append(outputs.cpu().numpy())
    #         true_coords.append(coords.cpu().numpy())

    # # 合并所有批次的预测结果和真实值
    # predicted_coords = np.concatenate(predicted_coords, axis=0)
    # true_coords = np.concatenate(true_coords, axis=0)
    # # 绘制散点图
    # plt.figure(figsize=(10, 6))
    # plt.scatter(
    #     true_coords[:, 0],
    #     true_coords[:, 1],
    #     label="True Coordinates",
    #     marker="o",
    #     s=30,
    #     alpha=0.7,
    # )
    # plt.scatter(
    #     predicted_coords[:, 0],
    #     predicted_coords[:, 1],
    #     label="Predicted Coordinates",
    #     marker="x",
    #     s=30,
    #     alpha=0.7,
    # )
    # plt.xlabel("X-coordinate")
    # plt.ylabel("Y-coordinate")
    # plt.legend()
    # plt.title("True vs. Predicted Coordinates")
    # plt.grid(True)
    # plt.text(
    #     0,
    #     0,
    #     f"Loss: {total_test_loss / len(test_loader)}",
    #     ha="left",
    #     va="bottom",
    #     transform=plt.gca().transAxes,
    # )
    # test_result_name = testing_results_folder + f"/epoch-{epoch}-test-result.png"
    # plt.savefig(test_result_name)
    
    # plt.show()
    # print(f"Test Loss: {total_test_loss / len(test_loader)}")
